# HoloAssist Per-Task Analysis

In [ ]:
# Read data
import pandas as pd

vlm_tsi_df = pd.read_csv(
    "../ego4d-goalstep+vlm-tsi+step=10492+use_narration+per_task.csv"
)
videollm_online_df = pd.read_csv(
    "../holo-assist+videollm-online+threshold=0.8+use_narration+per_task.csv"
)

In [ ]:
# print out tasks
assert set(vlm_tsi_df["task_type"].unique()) == set(
    videollm_online_df["task_type"].unique()
)
print(vlm_tsi_df["task_type"].unique())

In [ ]:
# group tasks
task_groups = {
    "Setup Electronics": [
        "setup camera",
        "setup switch",
        "setup big printer",
        "setup small printer",
        "setup gopro",
        "assemble laser scanner",
        "assemble computer",
    ],
    "Make Coffee": [
        "make coffee with nespresso machine",
        "make coffee with espresso machine",
    ],
    "Repair Machinery": ["change belt", "change circuit breaker", "fix motorcycle"],
    "Assemble Furniture": [
        "assemble nightstand",
        "assemble stool",
        "assemble tray table",
        "assemble utility cart",
    ],
    "Disassemble Furniture": [
        "disassemble nightstand",
        "disassemble stool",
        "disassemble tray table",
        "disassemble utility cart",
    ],
}

In [ ]:
# compute stats per task group
from pprint import pprint


def compute_stats(df):
    results = {}
    for group, tasks in task_groups.items():
        subset = df[df["task_type"].isin(tasks)]
        if subset.empty:
            continue
        results[group] = {
            "num_tasks": len(subset),
            "mean_semantic": float(subset["mean_acc_f1_adjusted"].mean()),
            "mean_timing": float(subset["mean_timing_f1_adjusted"].mean()),
            "TRACE": float(subset["final_score"].mean()),
        }
    return results


vlm_tsi_stats = compute_stats(vlm_tsi_df)
print("VLM-TSI")
pprint(vlm_tsi_stats)
videollm_online_stats = compute_stats(videollm_online_df)
print("VideoLLM-Online")
pprint(videollm_online_stats)

In [ ]:
# calculate VLM-TSI - VideoLLM-Online per task group
diff_per_task_group = {}
for task_group in vlm_tsi_stats:
    diff_per_task_group[task_group] = {}
    diff_per_task_group[task_group]["TRACE"] = (
        vlm_tsi_stats[task_group]["TRACE"] - videollm_online_stats[task_group]["TRACE"]
    )
    diff_per_task_group[task_group]["mean_semantic"] = (
        vlm_tsi_stats[task_group]["mean_semantic"]
        - videollm_online_stats[task_group]["mean_semantic"]
    )
    diff_per_task_group[task_group]["mean_timing"] = (
        vlm_tsi_stats[task_group]["mean_timing"]
        - videollm_online_stats[task_group]["mean_timing"]
    )
    diff_per_task_group[task_group]["num_tasks"] = vlm_tsi_stats[task_group][
        "num_tasks"
    ]
print("VLM-TSI - VideoLLM-Online")
pprint(diff_per_task_group)